In [2]:
import pandas as pd

col_name = ['idx','from','to','distance']
dis_df = pd.read_csv('USA-road-d.NY.gr',sep=' ',skiprows=7,names = col_name)
print(dis_df)
temp = dis_df.drop(['idx'],axis=1)
print(temp)
dis_list=temp.values.tolist()
print(dis_list[:10])

       idx    from      to  distance
0        a       1       2       803
1        a       2       1       803
2        a       3       4       158
3        a       4       3       158
4        a       5       6       774
...     ..     ...     ...       ...
733841   a  239390  239392      1557
733842   a  263466  263813       106
733843   a  263813  263466       106
733844   a  261228  259707       389
733845   a  259707  261228       389

[733846 rows x 4 columns]
          from      to  distance
0            1       2       803
1            2       1       803
2            3       4       158
3            4       3       158
4            5       6       774
...        ...     ...       ...
733841  239390  239392      1557
733842  263466  263813       106
733843  263813  263466       106
733844  261228  259707       389
733845  259707  261228       389

[733846 rows x 3 columns]
[[1, 2, 803], [2, 1, 803], [3, 4, 158], [4, 3, 158], [5, 6, 774], [6, 5, 774], [7, 8, 1531], [8, 7, 1531],

In [24]:
from py2neo import Graph, Node, Relationship
import heapq

g = Graph()
n = 264346

nodes = [Node("Node",name=str(i)) for i in range(n)] # 264346개의 노드 생성
tx = g.begin()
for i in tqdm(dis_list,mininterval=0.01):
    re = Relationship(nodes[i[0]],"DIST",nodes[i[1]],dist=str(i[2]))
    tx.create(re)
tx.commit()

if tx.finished()==False:
    print('transaction is still open')
else:
    print('transaction has been completed')

  3%|▎         | 22718/733846 [4:44:31<148:26:03,  1.33it/s]     


KeyboardInterrupt: 

In [22]:
df_to_dict = []

#0번 node부터 관계 값을 가져와 dataframe으로 저장
for i in range(n):
    df = g.run("match(a)-[r]->(b)"
              "where a.name = $name"
              "return a.name as node1, r.dist as dist, b.name as node2",
              {"name":str(i)}).to_data_frame()
    df_to_dict.append([])
    for j in range(len(df)):
        df_to_dict[i].append( (df.loc[j,'node2'],int(df.loc[j,'dist'])))
        
#dataframe을 dictionary로 변환
#dictionary 구조 : 기준 노드 {가리키는 노드1:두 노드의 거리1, 가리키는 노드2:두 노드의 거리2, ...}

dic = dict()
for i in range(n):
    dic.update({str(i):dict(df_to_dict[i])})

print(dic)
print(type(dic))
print(len(dic))